## Complete end to end code 
* [1. Setup](#1)
* [2. Algo Logic](#2)
* [3. Backtesting](#3)
* [4. Live Trading](#4)  

<a id='1'></a>
# 1. Set Up

In [ ]:
import alpaca_trade_api as tradeapi
import pandas as pd
import os
import json
from alpaca_trade_api.rest import REST, TimeFrame
import quantstats as qs
# # Replace these with your own API key pairs
# API_KEY_ID = 'PKJAGATSRN1M3JLPK3DO'
# API_SECRET_KEY = '5lc8SYlcqmsPzO9N97U66fAYcMavTuik9IR60oTQ'

# Connect to the Alpaca API

os.chdir("D:\\Python\\Alpaca_Keys") #change this based on the path on your local machine

endpoint = "https://data.alpaca.markets/v2"
headers = json.loads(open("keys.txt",'r').read())

In [55]:
api = REST(headers["APCA-API-KEY-ID"], headers["APCA-API-SECRET-KEY"], base_url=endpoint)
#api = tradeapi.REST(API_KEY_ID, API_SECRET_KEY, base_url='https://paper-api.alpaca.markets')

# Get historical price data for a stock
symbol = 'AAPL'
start_date = "2020-01-01"
end_date = "2020-12-31"
start_date

'2020-01-01'

In [56]:
barset = {}
barset[symbol] = api.get_bars(symbol, TimeFrame.Day, start=start_date, end=end_date, adjustment='all').df

In [57]:
data = barset[symbol]

In [67]:
data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2020-01-02 05:00:00+00:00,72.34,73.41,72.09,73.35,135650672,283487,72.88
2020-01-03 05:00:00+00:00,72.57,73.40,72.41,72.63,146535580,280941,72.97
2020-01-06 05:00:00+00:00,71.74,73.25,71.49,73.21,118579096,242866,72.72
2020-01-07 05:00:00+00:00,73.22,73.48,72.65,72.87,111510672,227869,73.00
2020-01-08 05:00:00+00:00,72.57,74.35,72.57,74.04,132363788,261771,73.55
...,...,...,...,...,...,...,...
2020-12-24 05:00:00+00:00,129.39,131.49,129.17,130.03,54930664,365719,130.38
2020-12-28 05:00:00+00:00,132.02,135.32,131.54,134.68,124488470,782402,133.84
2020-12-29 05:00:00+00:00,136.01,136.74,132.36,132.88,121047748,797015,134.37


<a id='2'></a>
# 2. Algo Logic

In [58]:
def stochastic_rsi_strategy(data):
    # Calculate the Stochastic Oscillator
    high = data['high'].rolling(window=14).max()
    low = data['low'].rolling(window=14).min()
    k = 100 * (data['close'] - low) / (high - low)
    d = k.rolling(window=3).mean()

    # Calculate the RSI
    delta = data['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    # Generate buy and sell signals
    buy_signal = (k > 80) & (rsi > 50)
    sell_signal = (k < 20) & (rsi < 50)

    # Combine the signals into a single DataFrame
    signals = pd.DataFrame(index=data.index)
    signals['buy'] = buy_signal
    signals['sell'] = sell_signal

    return signals

In [ ]:
<a id='3'></a>
# 3. Backtesting

In [59]:
# Generate trading signals
signals = stochastic_rsi_strategy(data)

# Backtest the strategy
initial_capital = 100000.0
position = 0
for i in range(len(data)):
    # Check for buy signals
    if signals.loc[data.index[i], 'buy']:
        shares_to_buy = int(initial_capital / data['close'][i])
        position += shares_to_buy
        initial_capital -= shares_to_buy * data['close'][i]

    # Check for sell signals
    if signals.loc[data.index[i], 'sell']:
        initial_capital += position * data['close'][i]
        position = 0

# Print the final portfolio value and P/L
final_portfolio_value = initial_capital + position * data['close'][-1]
pnl = final_portfolio_value - 100000.0
print(f'Final Portfolio Value: ${final_portfolio_value:.2f}')
print(f'P/L: ${pnl:.2f}')

Final Portfolio Value: $168305.97
P/L: $68305.97


In [60]:
qs.extend_pandas()
returns = data['close'].pct_change()
qs.reports.full(returns, 'AAPL Buy and Hold')

- AAPL BUY AND HOLD: No data found, symbol may be delisted


C:\Users\tatsa\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2343: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\tatsa\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2344: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\tatsa\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2343: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
C:\Users\tatsa\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2344: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and datetime

In [65]:
price

172.98

In [66]:
symbol = 'AAPL'
price = api.get_latest_trade(symbol).price

# Generate trading signals
signals = stochastic_rsi_strategy(price)

# Live trade the strategy
while True:
    # Get the current price of the stock
    price = api.get_latest_trade (symbol).price

    # Check for buy signals
    if signals.loc[0, 'buy']:
        api.submit_order(
            symbol=symbol,
            qty=1,
            side='buy',
            type='market',
            time_in_force='gtc'
        )

    # Check for sell signals
    if signals.loc[0, 'sell']:
        api.submit_order(
            symbol=symbol,
            qty=1,
            side='sell',
            type='market',
            time_in_force='gtc'
        )

    # Wait for the next trading interval
    time.sleep(60)

TypeError: 'float' object is not subscriptable